In [1]:
# Cell 1: Setup and Imports
"""
Chain of Thought Agent - Testing Notebook
==========================================
This notebook implements a Chain of Thought (CoT) agent that:
1. Plans the ENTIRE task upfront
2. Generates a detailed reasoning chain
3. Executes all steps sequentially
4. Compares performance vs ReAct pattern

Key Differences from ReAct:
- ReAct: Think → Act → Observe → Think → Act (iterative)
- CoT: Think (plan all steps) → Execute all → Report (batch)
"""

import os
import sys
import json
from pathlib import Path
from typing import List, Dict, Any
from dotenv import load_dotenv
from rich import print as rprint
from rich.console import Console
from rich.panel import Panel
from rich.syntax import Syntax
from rich.table import Table
from rich.progress import Progress, SpinnerColumn, TextColumn

# Add src to path
sys.path.append(str(Path.cwd().parent / 'src'))

# Load environment
load_dotenv()

console = Console()
console.print(Panel.fit("🧠 Chain of Thought Agent - Testing Environment", style="bold blue"))

╭─────────────────────────────────────────────────╮
│ 🧠 Chain of Thought Agent - Testing Environment │
╰─────────────────────────────────────────────────╯

In [2]:
# Cell 2: Configuration
"""
Agent Configuration
"""

CONFIG = {
    "llm_provider": os.getenv("LLM_PROVIDER", "ollama"),
    "ollama_model": os.getenv("OLLAMA_MODEL", "llama3.1:8b"),
    "max_iterations": int(os.getenv("MAX_ITERATIONS", 15)),
    "sandbox_dir": Path(os.getenv("SANDBOX_DIR", "../sandbox")).resolve(),
    "enable_shell": os.getenv("ENABLE_SHELL_EXECUTION", "true").lower() == "true",
    "log_level": os.getenv("LOG_LEVEL", "INFO"),
}

# Create sandbox directory
CONFIG["sandbox_dir"].mkdir(exist_ok=True)

rprint("\n[bold cyan]Configuration:[/bold cyan]")
for key, value in CONFIG.items():
    rprint(f"  • {key}: [yellow]{value}[/yellow]")

Configuration:

• llm_provider: ollama  # or openai, anthropic

• ollama_model: llama3.1:8b

• max_iterations: 15

• sandbox_dir: /Users/edsonflores/Documents/vsc/Freelance/dev-setup-agent/notebooks/sandbox

• enable_shell: True

• log_level: INFO

In [3]:
# Cell 3: Tool Interface Definition (Reused from agent_testing.ipynb)
"""
Base Tool Interface - All tools must implement this
"""

from abc import ABC, abstractmethod
from pydantic import BaseModel, Field
from enum import Enum
from typing import Union

class ToolStatus(str, Enum):
    SUCCESS = "success"
    ERROR = "error"
    PARTIAL = "partial"

class ToolResult(BaseModel):
    """Standardized tool execution result"""
    status: ToolStatus
    output: str
    error: Union[str, None] = None
    metadata: Dict[str, Any] = Field(default_factory=dict)
    
    def __str__(self):
        if self.status == ToolStatus.SUCCESS:
            return f"✅ {self.output}"
        elif self.status == ToolStatus.ERROR:
            return f"❌ Error: {self.error}"
        else:
            return f"⚠️  {self.output}"

class BaseTool(ABC):
    """Base class for all agent tools"""
    
    @property
    @abstractmethod
    def name(self) -> str:
        """Tool name for the LLM"""
        pass
    
    @property
    @abstractmethod
    def description(self) -> str:
        """What this tool does"""
        pass
    
    @property
    @abstractmethod
    def parameters_schema(self) -> Dict:
        """JSON schema for tool parameters"""
        pass
    
    @abstractmethod
    def execute(self, **kwargs) -> ToolResult:
        """Execute the tool with given parameters"""
        pass
    
    def to_dict(self) -> Dict:
        """Convert tool to LLM-friendly format"""
        return {
            "name": self.name,
            "description": self.description,
            "parameters": self.parameters_schema
        }

rprint("\n[bold green]✓ Tool interface defined[/bold green]")

✓ Tool interface defined

In [4]:
# Cell 4-6: Import Tool Implementations from agent_testing.ipynb
"""
For this notebook, we'll reuse the same tool implementations.
Run cells 4-6 from agent_testing.ipynb to get:
- FileSystemTool
- ShellTool  
- CodeGeneratorTool
"""

import subprocess
from typing import Set

# FileSystemTool
class FileSystemTool(BaseTool):
    def __init__(self, sandbox_dir: Path):
        self.sandbox_dir = sandbox_dir
    
    @property
    def name(self) -> str:
        return "file_system"
    
    @property
    def description(self) -> str:
        return """Perform file system operations (create, read, write, list files).
        All operations are scoped to the sandbox directory for safety.
        
        Supported operations:
        - create: Create a new file with content
        - read: Read file contents
        - write: Write/overwrite file contents
        - list: List files in a directory
        - mkdir: Create a directory
        """
    
    @property
    def parameters_schema(self) -> Dict:
        return {
            "type": "object",
            "properties": {
                "operation": {
                    "type": "string",
                    "enum": ["create", "read", "write", "list", "mkdir"],
                    "description": "The file operation to perform"
                },
                "path": {
                    "type": "string",
                    "description": "Relative path within sandbox"
                },
                "content": {
                    "type": "string",
                    "description": "File content (for create/write operations)"
                }
            },
            "required": ["operation", "path"]
        }
    
    def _resolve_path(self, path: str) -> Path:
        full_path = (self.sandbox_dir / path).resolve()
        if not str(full_path).startswith(str(self.sandbox_dir)):
            raise ValueError(f"Path {path} is outside sandbox!")
        return full_path
    
    def execute(self, operation: str, path: str, content: str = None) -> ToolResult:
        try:
            full_path = self._resolve_path(path)
            
            if operation == "create":
                if full_path.exists():
                    return ToolResult(status=ToolStatus.ERROR, output="", error=f"File {path} already exists")
                full_path.parent.mkdir(parents=True, exist_ok=True)
                full_path.write_text(content or "")
                return ToolResult(status=ToolStatus.SUCCESS, output=f"Created file: {path}", 
                                metadata={"path": str(full_path), "size": len(content or "")})
            
            elif operation == "read":
                if not full_path.exists():
                    return ToolResult(status=ToolStatus.ERROR, output="", error=f"File {path} does not exist")
                content = full_path.read_text()
                return ToolResult(status=ToolStatus.SUCCESS, output=content,
                                metadata={"path": str(full_path), "size": len(content)})
            
            elif operation == "write":
                full_path.parent.mkdir(parents=True, exist_ok=True)
                full_path.write_text(content or "")
                return ToolResult(status=ToolStatus.SUCCESS, output=f"Wrote to file: {path}",
                                metadata={"path": str(full_path), "size": len(content or "")})
            
            elif operation == "list":
                if not full_path.exists():
                    return ToolResult(status=ToolStatus.ERROR, output="", error=f"Directory {path} does not exist")
                files = [f.name for f in full_path.iterdir()]
                return ToolResult(status=ToolStatus.SUCCESS, output="\n".join(files),
                                metadata={"count": len(files), "files": files})
            
            elif operation == "mkdir":
                full_path.mkdir(parents=True, exist_ok=True)
                return ToolResult(status=ToolStatus.SUCCESS, output=f"Created directory: {path}",
                                metadata={"path": str(full_path)})
            
            else:
                return ToolResult(status=ToolStatus.ERROR, output="", error=f"Unknown operation: {operation}")
        
        except Exception as e:
            return ToolResult(status=ToolStatus.ERROR, output="", error=str(e))

# ShellTool
class ShellTool(BaseTool):
    def __init__(self, sandbox_dir: Path, allowed_commands: Set[str] = None):
        self.sandbox_dir = sandbox_dir
        self.allowed_commands = allowed_commands or {
            'ls', 'cat', 'echo', 'pwd', 'mkdir', 'touch',
            'git', 'python', 'pip', 'npm', 'poetry'
        }
    
    @property
    def name(self) -> str:
        return "shell"
    
    @property
    def description(self) -> str:
        return f"""Execute shell commands in the sandbox directory.
        Allowed commands: {', '.join(sorted(self.allowed_commands))}
        """
    
    @property
    def parameters_schema(self) -> Dict:
        return {
            "type": "object",
            "properties": {
                "command": {"type": "string", "description": "Shell command to execute"},
                "working_dir": {"type": "string", "description": "Working directory relative to sandbox"}
            },
            "required": ["command"]
        }
    
    def _is_safe_command(self, command: str) -> bool:
        base_cmd = command.split()[0]
        return base_cmd in self.allowed_commands
    
    def execute(self, command: str, working_dir: str = ".") -> ToolResult:
        if not self._is_safe_command(command):
            return ToolResult(status=ToolStatus.ERROR, output="",
                            error=f"Command '{command.split()[0]}' not allowed")
        
        try:
            cwd = self.sandbox_dir / working_dir
            result = subprocess.run(command, shell=True, cwd=cwd, capture_output=True, text=True, timeout=30)
            
            if result.returncode == 0:
                return ToolResult(status=ToolStatus.SUCCESS, 
                                output=result.stdout or "Command executed successfully",
                                metadata={"returncode": result.returncode, "stderr": result.stderr})
            else:
                return ToolResult(status=ToolStatus.ERROR, output=result.stdout, error=result.stderr,
                                metadata={"returncode": result.returncode})
        except subprocess.TimeoutExpired:
            return ToolResult(status=ToolStatus.ERROR, output="", error="Command timed out (30s limit)")
        except Exception as e:
            return ToolResult(status=ToolStatus.ERROR, output="", error=str(e))

# CodeGeneratorTool
class CodeGeneratorTool(BaseTool):
    TEMPLATES = {
        "fastapi_main": '''"""\nFastAPI Application Entry Point\n"""\nfrom fastapi import FastAPI, HTTPException\nfrom fastapi.middleware.cors import CORSMiddleware\nfrom pydantic import BaseModel\n\napp = FastAPI(\n    title="{project_name}",\n    description="{description}",\n    version="0.1.0"\n)\n\n# CORS middleware\napp.add_middleware(\n    CORSMiddleware,\n    allow_origins=["*"],\n    allow_credentials=True,\n    allow_methods=["*"],\n    allow_headers=["*"],\n)\n\n@app.get("/")\nasync def root():\n    return {{"message": "Welcome to {project_name}"}}\n\n@app.get("/health")\nasync def health_check():\n    return {{"status": "healthy"}}\n\nif __name__ == "__main__":\n    import uvicorn\n    uvicorn.run(app, host="0.0.0.0", port=8000)\n''',
        "requirements": '''# Core dependencies\nfastapi==0.104.1\nuvicorn[standard]==0.24.0\npydantic==2.5.0\npython-dotenv==1.0.0\n\n# Testing\npytest==7.4.3\nhttpx==0.25.2\n''',
        "gitignore": '''# Python\n__pycache__/\n*.py[cod]\n*$py.class\n.Python\nvenv/\n.venv/\nENV/\n\n# IDE\n.vscode/\n.idea/\n\n# Environment\n.env\n.env.local\n'''
    }
    
    @property
    def name(self) -> str:
        return "code_generator"
    
    @property
    def description(self) -> str:
        return """Generate code from templates (FastAPI, Dockerfile, configs, etc.).
        Available templates: fastapi_main, requirements, gitignore
        """
    
    @property
    def parameters_schema(self) -> Dict:
        return {
            "type": "object",
            "properties": {
                "template": {
                    "type": "string",
                    "enum": list(self.TEMPLATES.keys()),
                    "description": "Template to generate"
                },
                "variables": {
                    "type": "object",
                    "description": "Variables for template substitution"
                }
            },
            "required": ["template"]
        }
    
    def execute(self, template: str, variables: Dict[str, str] = None) -> ToolResult:
        if template not in self.TEMPLATES:
            return ToolResult(status=ToolStatus.ERROR, output="",
                            error=f"Template '{template}' not found")
        try:
            code = self.TEMPLATES[template]
            if variables:
                code = code.format(**variables)
            return ToolResult(status=ToolStatus.SUCCESS, output=code,
                            metadata={"template": template, "lines": len(code.split('\n'))})
        except KeyError as e:
            return ToolResult(status=ToolStatus.ERROR, output="", error=f"Missing variable: {e}")
        except Exception as e:
            return ToolResult(status=ToolStatus.ERROR, output="", error=str(e))

# Initialize tools
fs_tool = FileSystemTool(CONFIG["sandbox_dir"])
shell_tool = ShellTool(CONFIG["sandbox_dir"])
code_gen_tool = CodeGeneratorTool()

rprint("\n[bold green]✓ All tools initialized[/bold green]")

✓ All tools initialized

In [5]:
# Cell 7: LLM Integration (Enhanced with Error Handling)
"""
LLM Integration - Connect to Ollama with robust error handling
"""

import httpx
from typing import Optional, Dict

class OllamaClient:
    """
    Ollama LLM Client with comprehensive error handling and logging
    """

    def __init__(self, base_url: str = "http://localhost:11434", model: str = "llama3.1:8b"):
        self.base_url = base_url
        self.model = model
        self.client = httpx.Client(timeout=60.0)
        self.last_error: Optional[str] = None

    def generate(self, prompt: str, system: str = None) -> str:
        """
        Generate completion from Ollama

        Returns:
            Generated text or error message
        """
        messages = []
        if system:
            messages.append({"role": "system", "content": system})
        messages.append({"role": "user", "content": prompt})

        try:
            rprint(f"[dim]→ Sending request to {self.base_url} (model: {self.model})[/dim]")

            response = self.client.post(
                f"{self.base_url}/api/chat",
                json={
                    "model": self.model,
                    "messages": messages,
                    "stream": False,
                    "options": {"temperature": 0.1}
                }
            )

            # Check for HTTP errors
            response.raise_for_status()

            # Parse response
            result = response.json()

            # Validate response structure
            if "message" not in result:
                self.last_error = f"Invalid response format: missing 'message' field"
                rprint(f"[red]✗ {self.last_error}[/red]")
                rprint(f"[dim]Response: {result}[/dim]")
                return f"LLM Error: {self.last_error}"

            if "content" not in result["message"]:
                self.last_error = f"Invalid response format: missing 'content' field"
                rprint(f"[red]✗ {self.last_error}[/red]")
                return f"LLM Error: {self.last_error}"

            content = result["message"]["content"]
            rprint(f"[dim]✓ Received {len(content)} characters[/dim]")
            return content

        except httpx.HTTPStatusError as e:
            self.last_error = f"HTTP {e.response.status_code}: {e.response.text}"
            rprint(f"[red]✗ HTTP Error: {self.last_error}[/red]")
            return f"LLM Error: {self.last_error}"

        except httpx.ConnectError as e:
            self.last_error = f"Connection failed to {self.base_url}. Is Ollama running?"
            rprint(f"[red]✗ Connection Error: {self.last_error}[/red]")
            rprint(f"[yellow]Hint: Start Ollama with 'ollama serve'[/yellow]")
            return f"LLM Error: {self.last_error}"

        except httpx.TimeoutException as e:
            self.last_error = f"Request timed out after 60s"
            rprint(f"[red]✗ Timeout Error: {self.last_error}[/red]")
            return f"LLM Error: {self.last_error}"

        except httpx.ReadError as e:
            self.last_error = f"Failed to read response: {str(e)}"
            rprint(f"[red]✗ Read Error: {self.last_error}[/red]")
            return f"LLM Error: {self.last_error}"

        except KeyError as e:
            self.last_error = f"Missing expected field in response: {e}"
            rprint(f"[red]✗ Parse Error: {self.last_error}[/red]")
            return f"LLM Error: {self.last_error}"

        except Exception as e:
            self.last_error = f"Unexpected error: {type(e).__name__}: {str(e)}"
            rprint(f"[red]✗ Unexpected Error: {self.last_error}[/red]")
            return f"LLM Error: {self.last_error}"

    def test_connection(self) -> bool:
        """
        Test connection to Ollama server

        Returns:
            True if connection successful, False otherwise
        """
        try:
            rprint(f"[dim]Testing connection to {self.base_url}...[/dim]")
            response = self.client.get(f"{self.base_url}/api/tags", timeout=5.0)

            if response.status_code == 200:
                # Check if our model is available
                data = response.json()
                models = [m.get("name", "") for m in data.get("models", [])]

                if models:
                    rprint(f"[dim]Available models: {', '.join(models[:3])}{'...' if len(models) > 3 else ''}[/dim]")

                # Check if our specific model exists
                model_exists = any(self.model in m for m in models)
                if not model_exists:
                    rprint(f"[yellow]⚠ Warning: Model '{self.model}' not found in available models[/yellow]")
                    rprint(f"[yellow]  You may need to run: ollama pull {self.model}[/yellow]")

                return True
            else:
                self.last_error = f"Server returned status {response.status_code}"
                rprint(f"[red]✗ {self.last_error}[/red]")
                return False

        except httpx.ConnectError:
            self.last_error = f"Cannot connect to {self.base_url}"
            rprint(f"[red]✗ Connection failed: {self.last_error}[/red]")
            return False

        except httpx.TimeoutException:
            self.last_error = "Connection timeout"
            rprint(f"[red]✗ {self.last_error}[/red]")
            return False

        except Exception as e:
            self.last_error = f"{type(e).__name__}: {str(e)}"
            rprint(f"[red]✗ Error: {self.last_error}[/red]")
            return False

    def get_last_error(self) -> Optional[str]:
        """Get the last error message"""
        return self.last_error

# Initialize LLM client
rprint("\n[bold cyan]Initializing LLM Client...[/bold cyan]")
llm = OllamaClient(model=CONFIG["ollama_model"])

# Test connection with detailed diagnostics
if llm.test_connection():
    rprint("[green]✓ Ollama is running and accessible[/green]")

    # Optional: Test a simple generation
    rprint("\n[dim]Testing generation...[/dim]")
    test_response = llm.generate("Say 'OK' if you can hear me.")
    if not test_response.startswith("LLM Error"):
        rprint(f"[green]✓ Generation test passed[/green]")
    else:
        rprint(f"[yellow]⚠ Generation test failed: {test_response}[/yellow]")
else:
    rprint("[red]✗ Ollama not running or not accessible![/red]")
    rprint("[yellow]Troubleshooting steps:[/yellow]")
    rprint("  1. Start Ollama: ollama serve")
    rprint(f"  2. Verify server: curl {llm.base_url}/api/tags")
    rprint(f"  3. Pull model: ollama pull {llm.model}")
    if llm.last_error:
        rprint(f"\n[red]Last error: {llm.last_error}[/red]")

rprint("\n[bold green]✓ LLM integration ready[/bold green]")

Initializing LLM Client...

Testing connection to http://localhost:11434...

Available models: llama3.1:8b

✓ Ollama is running and accessible

Testing generation...

→ Sending request to http://localhost:11434 (model: llama3.1:8b)

✓ Received 136 characters

✓ Generation test passed

✓ LLM integration ready

In [6]:
# Cell 8: Chain of Thought Agent Implementation (Fixed JSON Parsing)
"""
Chain of Thought Agent - Plans everything upfront, then executes
"""

from dataclasses import dataclass
from datetime import datetime
import re

@dataclass
class PlannedAction:
    """A single planned action"""
    step_number: int
    reasoning: str
    tool: str
    parameters: Dict[str, Any]

@dataclass 
class ExecutedStep:
    """A step that has been executed"""
    planned_action: PlannedAction
    result: ToolResult
    timestamp: datetime

class ChainOfThoughtAgent:
    """
    Chain of Thought Agent
    
    Unlike ReAct which iterates (Think→Act→Observe→Think→Act...),
    CoT does:
    1. Think: Generate complete plan with all steps
    2. Execute: Run all actions sequentially
    3. Report: Summarize results
    
    Advantages:
    - Fewer LLM calls (1 planning call vs N iterations)
    - Faster execution
    - Better for deterministic tasks
    
    Disadvantages:
    - Can't adapt to unexpected results
    - Fails if initial plan is wrong
    - Less flexible than ReAct
    """
    
    def __init__(
        self,
        llm: OllamaClient,
        tools: List[BaseTool],
        verbose: bool = True
    ):
        self.llm = llm
        self.tools = {tool.name: tool for tool in tools}
        self.verbose = verbose
        self.plan: List[PlannedAction] = []
        self.execution_history: List[ExecutedStep] = []
        self.context = {}  # Store generated content
    
    def _build_planning_prompt(self) -> str:
        """Build system prompt for planning phase"""
        tools_desc = "\n\n".join([
            f"Tool: {tool.name}\n{tool.description}\nParameters: {json.dumps(tool.parameters_schema, indent=2)}"
            for tool in self.tools.values()
        ])
        
        return f"""You are a development setup agent using Chain of Thought reasoning.

Your task is to create a COMPLETE PLAN upfront before any execution.

Available Tools:
{tools_desc}

PLANNING RULES:
1. Think through the ENTIRE task from start to finish
2. Break down into discrete, sequential steps
3. Each step must use exactly ONE tool
4. Be explicit about dependencies (e.g., create directory before creating file)
5. When generating code, you MUST immediately follow with a step to write it
6. Plan for potential issues (e.g., file might exist)

Response Format:

REASONING:
[Your overall reasoning about the task and approach]

PLAN:

Step 1:
Reasoning: [Why this step is needed]
Tool: [tool_name]
Parameters: {{"param1": "value1", "param2": "value2"}}

Step 2:
Reasoning: [Why this step is needed]
Tool: [tool_name]
Parameters: {{"param1": "value1"}}

...(continue for all steps)

IMPORTANT:
- When using code_generator, the next step MUST be file_system to write that code
- Use PLACEHOLDER_CODE_FROM_STEP_N to reference code from a previous generation step
- Always create directories before files in them
- Be thorough - include verification steps (like listing directory)
- Keep JSON parameters on a single line or properly formatted

Example:
Step 1:
Reasoning: Create the project root directory first
Tool: file_system
Parameters: {{"operation": "mkdir", "path": "my_project"}}

Step 2:
Reasoning: Generate the FastAPI main.py code
Tool: code_generator
Parameters: {{"template": "fastapi_main", "variables": {{"project_name": "MyAPI", "description": "Test API"}}}}

Step 3:
Reasoning: Write the generated code to main.py file
Tool: file_system
Parameters: {{"operation": "create", "path": "my_project/main.py", "content": "PLACEHOLDER_CODE_FROM_STEP_2"}}
"""
    
    def _parse_plan(self, response: str) -> List[PlannedAction]:
        """Parse LLM response into structured plan with improved JSON handling"""
        plan = []
        
        # Extract the PLAN section
        plan_match = re.search(r'PLAN:\s*(.+)', response, re.DOTALL | re.IGNORECASE)
        if not plan_match:
            if self.verbose:
                console.print("[red]Could not find PLAN section in response[/red]")
            return plan
        
        plan_text = plan_match.group(1)
        
        # Split by "Step N:" to get individual steps
        step_splits = re.split(r'(?=Step\s+\d+:)', plan_text, flags=re.IGNORECASE)
        
        for step_block in step_splits:
            if not step_block.strip():
                continue
            
            # Extract step number
            step_num_match = re.search(r'Step\s+(\d+):', step_block, re.IGNORECASE)
            if not step_num_match:
                continue
            
            step_num = int(step_num_match.group(1))
            
            # Extract reasoning
            reasoning_match = re.search(r'Reasoning:\s*(.+?)\s*(?=Tool:)', step_block, re.DOTALL | re.IGNORECASE)
            reasoning = reasoning_match.group(1).strip() if reasoning_match else ""
            
            # Extract tool
            tool_match = re.search(r'Tool:\s*(\S+)', step_block, re.IGNORECASE)
            if not tool_match:
                if self.verbose:
                    console.print(f"[yellow]Warning: No tool found for step {step_num}[/yellow]")
                continue
            
            tool = tool_match.group(1).strip()
            
            # Extract parameters - IMPROVED MULTI-LINE JSON HANDLING
            params_match = re.search(r'Parameters:\s*(\{.*)', step_block, re.DOTALL | re.IGNORECASE)
            if not params_match:
                if self.verbose:
                    console.print(f"[yellow]Warning: No parameters found for step {step_num}[/yellow]")
                parameters = {}
            else:
                params_str = params_match.group(1).strip()
                
                # Find the JSON object - handle nested braces
                brace_count = 0
                json_end = 0
                in_string = False
                escape_next = False
                
                for i, char in enumerate(params_str):
                    if escape_next:
                        escape_next = False
                        continue
                    
                    if char == '\\':
                        escape_next = True
                        continue
                    
                    if char == '"' and not escape_next:
                        in_string = not in_string
                    
                    if not in_string:
                        if char == '{':
                            brace_count += 1
                        elif char == '}':
                            brace_count -= 1
                            if brace_count == 0:
                                json_end = i + 1
                                break
                
                if json_end > 0:
                    json_str = params_str[:json_end]
                else:
                    json_str = params_str
                
                # Try to parse JSON
                try:
                    parameters = json.loads(json_str)
                except json.JSONDecodeError as e:
                    if self.verbose:
                        console.print(f"[yellow]Warning: Could not parse parameters for step {step_num}[/yellow]")
                        console.print(f"[dim]  Error: {e}[/dim]")
                        console.print(f"[dim]  JSON string: {json_str[:100]}...[/dim]")
                    parameters = {}
            
            plan.append(PlannedAction(
                step_number=step_num,
                reasoning=reasoning,
                tool=tool,
                parameters=parameters
            ))
        
        return plan
    
    def _execute_plan(self) -> List[ExecutedStep]:
        """Execute the planned actions sequentially"""
        executed_steps = []
        
        with Progress(
            SpinnerColumn(),
            TextColumn("[progress.description]{task.description}"),
            console=console,
            transient=True
        ) as progress:
            task = progress.add_task("[cyan]Executing plan...", total=len(self.plan))
            
            for action in self.plan:
                if self.verbose:
                    progress.update(task, description=f"[cyan]Step {action.step_number}: {action.tool}")
                
                # Replace placeholders with actual generated content
                parameters = action.parameters.copy()
                if "content" in parameters:
                    content = str(parameters["content"])
                    # Check for PLACEHOLDER_CODE_FROM_STEP_N pattern
                    placeholder_match = re.search(r'PLACEHOLDER_CODE_FROM_STEP_(\d+)', content)
                    if placeholder_match:
                        ref_step = int(placeholder_match.group(1))
                        if ref_step in self.context:
                            parameters["content"] = self.context[ref_step]
                            if self.verbose:
                                console.print(f"[blue]  ↳ Replaced placeholder with code from step {ref_step}[/blue]")
                        else:
                            console.print(f"[yellow]  ⚠ No content found from step {ref_step}[/yellow]")
                
                # Execute tool
                if action.tool not in self.tools:
                    result = ToolResult(
                        status=ToolStatus.ERROR,
                        output="",
                        error=f"Tool '{action.tool}' not found"
                    )
                else:
                    try:
                        tool = self.tools[action.tool]
                        result = tool.execute(**parameters)
                        
                        # Store generated content for future steps
                        if action.tool == "code_generator" and result.status == ToolStatus.SUCCESS:
                            self.context[action.step_number] = result.output
                    
                    except TypeError as e:
                        result = ToolResult(
                            status=ToolStatus.ERROR,
                            output="",
                            error=f"Parameter error: {str(e)}. Parameters: {parameters}"
                        )
                    except Exception as e:
                        result = ToolResult(
                            status=ToolStatus.ERROR,
                            output="",
                            error=f"Execution error: {str(e)}"
                        )
                
                step = ExecutedStep(
                    planned_action=action,
                    result=result,
                    timestamp=datetime.now()
                )
                executed_steps.append(step)
                
                if self.verbose:
                    status_icon = "✅" if result.status == ToolStatus.SUCCESS else "❌"
                    console.print(f"  {status_icon} Step {action.step_number}: {result.status}")
                    if result.status == ToolStatus.ERROR:
                        console.print(f"    [red]Error: {result.error}[/red]")
                
                progress.advance(task)
        
        return executed_steps
    
    def run(self, task: str) -> str:
        """Execute task using Chain of Thought"""
        
        if self.verbose:
            console.print(Panel(f"[bold blue]Task:[/bold blue] {task}", expand=False))
            console.print("\n[bold yellow]Phase 1: Planning[/bold yellow]")
        
        # Phase 1: Planning
        planning_prompt = f"""Task: {task}

Create a complete step-by-step plan to accomplish this task.
Think carefully about the order of operations and dependencies.
"""
        
        response = self.llm.generate(
            prompt=planning_prompt,
            system=self._build_planning_prompt()
        )
        
        if self.verbose:
            console.print(f"\n[dim]{response}[/dim]\n")
        
        # Parse the plan
        self.plan = self._parse_plan(response)
        
        if not self.plan:
            return "Failed to create a valid plan"
        
        if self.verbose:
            table = Table(title=f"Plan: {len(self.plan)} Steps")
            table.add_column("Step", style="cyan")
            table.add_column("Tool", style="green")
            table.add_column("Reasoning", style="yellow")
            
            for action in self.plan:
                reasoning_short = action.reasoning[:60] + "..." if len(action.reasoning) > 60 else action.reasoning
                table.add_row(str(action.step_number), action.tool, reasoning_short)
            
            console.print(table)
        
        # Phase 2: Execution
        if self.verbose:
            console.print("\n[bold yellow]Phase 2: Execution[/bold yellow]\n")
        
        self.execution_history = self._execute_plan()
        
        # Phase 3: Report
        success_count = sum(1 for s in self.execution_history if s.result.status == ToolStatus.SUCCESS)
        total_count = len(self.execution_history)
        
        if success_count == total_count:
            result = f"✓ Task completed successfully! All {total_count} steps executed."
        else:
            result = f"⚠ Task partially completed: {success_count}/{total_count} steps succeeded."
        
        if self.verbose:
            console.print(Panel(f"[bold green]{result}[/bold green]", expand=False))
        
        return result
    
    def get_summary(self) -> Dict:
        """Get execution summary"""
        return {
            "total_planned_steps": len(self.plan),
            "total_executed_steps": len(self.execution_history),
            "tools_used": list(set(s.planned_action.tool for s in self.execution_history)),
            "success_rate": sum(1 for s in self.execution_history if s.result.status == ToolStatus.SUCCESS) / len(self.execution_history) if self.execution_history else 0,
            "steps": [
                {
                    "step": s.planned_action.step_number,
                    "tool": s.planned_action.tool,
                    "status": s.result.status,
                    "reasoning": s.planned_action.reasoning
                }
                for s in self.execution_history
            ]
        }

rprint("\n[bold green]✓ Chain of Thought Agent implemented with improved JSON parsing[/bold green]")

✓ Chain of Thought Agent implemented with improved JSON parsing

In [7]:
# Cell 9: Test Chain of Thought Agent
"""
Test the CoT Agent with a FastAPI project creation task
"""

import shutil

# Clean up any existing project
if (CONFIG["sandbox_dir"] / "cot_fastapi_project").exists():
    shutil.rmtree(CONFIG["sandbox_dir"] / "cot_fastapi_project")

# Initialize CoT agent
cot_agent = ChainOfThoughtAgent(
    llm=llm,
    tools=[fs_tool, shell_tool, code_gen_tool],
    verbose=True
)

# Task
task = """Create a FastAPI project named 'cot_fastapi_project' with:
- main.py (use template with project_name='CoT API', description='Built with Chain of Thought')
- requirements.txt
- .gitignore

Then list the directory contents to verify."""

# Run the agent
result = cot_agent.run(task)

# Show summary
console.print("\n" + "="*60)
console.print("[bold cyan]Execution Summary:[/bold cyan]")
summary = cot_agent.get_summary()

table = Table(title="CoT Agent Execution")
table.add_column("Step", style="cyan")
table.add_column("Tool", style="green")
table.add_column("Status", style="yellow")
table.add_column("Reasoning", style="white")

for step_info in summary["steps"]:
    status_icon = "✅" if step_info["status"] == "success" else "❌"
    reasoning_short = step_info["reasoning"][:40] + "..." if len(step_info["reasoning"]) > 40 else step_info["reasoning"]
    table.add_row(
        str(step_info["step"]),
        step_info["tool"],
        f"{status_icon} {step_info['status']}",
        reasoning_short
    )

console.print(table)
console.print(f"\n[bold]Tools Used:[/bold] {', '.join(summary['tools_used'])}")
console.print(f"[bold]Success Rate:[/bold] {summary['success_rate']:.1%}")
console.print(f"[bold]Total Steps:[/bold] {summary['total_executed_steps']}")

╭─────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Task: Create a FastAPI project named 'cot_fastapi_project' with:                                │
│ - main.py (use template with project_name='CoT API', description='Built with Chain of Thought') │
│ - requirements.txt                                                                              │
│ - .gitignore                                                                                    │
│                                                                                                 │
│ Then list the directory contents to verify.                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────╯

Phase 1: Planning

→ Sending request to http://localhost:11434 (model: llama3.1:8b)

✓ Received 2044 characters

REASONING:
To create a FastAPI project, we need to generate code for main.py, requirements.txt, and .gitignore files. We'll 
use the code_generator tool to generate these files based on templates. After generating each file, we'll 
immediately write it using the file_system tool. Finally, we'll list the directory contents to verify that 
everything is in place.

PLAN:

Step 1:
Reasoning: Create the project root directory first
Tool: file_system
Parameters: {"operation": "mkdir", "path": "cot_fastapi_project"}

Step 2:
Reasoning: Generate the FastAPI main.py code using the fastapi_main template
Tool: code_generator
Parameters: {"template": "fastapi_main", "variables": {"project_name": "CoT API", "description": "Built with Chain 
of Thought"}}

Step 3:
Reasoning: Write the generated code to main.py file
Tool: file_system
Parameters: {"operation": "create", "path": "cot_fastapi_project/main.py", "content": 
"PLACEHOLDER_CODE_FROM_STEP_2"}

Step 4:
Reasoning: Generate the requirements.txt file using the requirements template
Tool: code_generator
Parameters: {"template": "requirements", "variables": {}} (no variables needed for this template)

Step 5:
Reasoning: Write the generated code to requirements.txt file
Tool: file_system
Parameters: {"operation": "create", "path": "cot_fastapi_project/requirements.txt", "content": 
"PLACEHOLDER_CODE_FROM_STEP_4"}

Step 6:
Reasoning: Generate the .gitignore file using the gitignore template
Tool: code_generator
Parameters: {"template": "gitignore", "variables": {}} (no variables needed for this template)

Step 7:
Reasoning: Write the generated code to .gitignore file
Tool: file_system
Parameters: {"operation": "create", "path": "cot_fastapi_project/.gitignore", "content": 
"PLACEHOLDER_CODE_FROM_STEP_6"}

Step 8:
Reasoning: List the directory contents to verify that everything is in place
Tool: file_system
Parameters: {"operation": "list", "path": "cot_fastapi_project"}

Note: The PLACEHOLDER_CODE_FROM_STEP_N syntax will be used to reference the generated code from previous steps.

...

Execution Summary:

                                CoT Agent Execution                                 
┏━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Step ┃ Tool           ┃ Status     ┃ Reasoning                                   ┃
┡━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1    │ file_system    │ ✅ success │ Create the project root directory first     │
│ 2    │ code_generator │ ✅ success │ Generate the FastAPI main.py code using ... │
│ 3    │ file_system    │ ✅ success │ Write the generated code to main.py file    │
│ 4    │ code_generator │ ✅ success │ Generate the requirements.txt file using... │
│ 5    │ file_system    │ ✅ success │ Write the generated code to requirements... │
│ 6    │ code_generator │ ✅ success │ Generate the .gitignore file using the g... │
│ 7    │ file_system    │ ✅ success │ Write the generated code to .gitignore f... │
│ 8    │ file_system    │ ✅ success │ List the directory contents to verify th... │
└──────┴────────────────┴────────────┴─────────────────────────────────────────────┘

Tools Used: code_generator, file_system

Success Rate: 100.0%

Total Steps: 8

In [19]:
# Cell 10: Compare CoT vs ReAct
"""
Compare Chain of Thought vs ReAct patterns
"""

comparison_table = Table(title="Chain of Thought vs ReAct Comparison")
comparison_table.add_column("Aspect", style="cyan", width=20)
comparison_table.add_column("Chain of Thought", style="blue", width=35)
comparison_table.add_column("ReAct", style="magenta", width=35)

comparisons = [
    ("Pattern", "Think (all steps) → Execute all → Report", "Think → Act → Observe → Think → Act..."),
    ("LLM Calls", "1 planning call + execution", "N calls (one per iteration)"),
    ("Speed", "Faster (fewer LLM calls)", "Slower (multiple LLM calls)"),
    ("Adaptability", "Cannot adapt to errors", "Adapts based on observations"),
    ("Best For", "Deterministic, well-defined tasks", "Complex, uncertain tasks"),
    ("Failure Mode", "Fails if plan is wrong", "Can recover from errors"),
    ("Complexity", "Simpler implementation", "More complex loop logic"),
    ("Cost", "Lower (fewer API calls)", "Higher (more API calls)"),
    ("Debugging", "Easier (plan visible upfront)", "Harder (evolves over time)"),
    ("Context Size", "Smaller context per call", "Growing context each iteration")
]

for aspect, cot, react in comparisons:
    comparison_table.add_row(aspect, cot, react)

console.print("\n")
console.print(comparison_table)

console.print("\n[bold yellow]When to Use Each:[/bold yellow]")
console.print("\n[bold blue]Chain of Thought:[/bold blue]")
console.print("  • Setting up standard project structures")
console.print("  • Generating boilerplate code")
console.print("  • Tasks with clear, sequential steps")
console.print("  • When cost/speed matters")
console.print("\n[bold magenta]ReAct:[/bold magenta]")
console.print("  • Debugging unknown issues")
console.print("  • Exploratory tasks")
console.print("  • Tasks requiring adaptation")
console.print("  • Complex problem-solving")

                                Chain of Thought vs ReAct Comparison                                
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Aspect               ┃ Chain of Thought                    ┃ ReAct                               ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Pattern              │ Think (all steps) → Execute all →   │ Think → Act → Observe → Think →     │
│                      │ Report                              │ Act...                              │
│ LLM Calls            │ 1 planning call + execution         │ N calls (one per iteration)         │
│ Speed                │ Faster (fewer LLM calls)            │ Slower (multiple LLM calls)         │
│ Adaptability         │ Cannot adapt to errors              │ Adapts based on observations        │
│ Best For             │ Deterministic, well-defined tasks   │ Complex, uncertain tasks            │
│ Failure Mode         │ Fails if plan is wrong              │ Can recover from errors             │
│ Complexity           │ Simpler implementation              │ More complex loop logic             │
│ Cost                 │ Lower (fewer API calls)             │ Higher (more API calls)             │
│ Debugging            │ Easier (plan visible upfront)       │ Harder (evolves over time)          │
│ Context Size         │ Smaller context per call            │ Growing context each iteration      │
└──────────────────────┴─────────────────────────────────────┴─────────────────────────────────────┘

When to Use Each:

Chain of Thought:

• Setting up standard project structures

• Generating boilerplate code

...

ReAct:

• Debugging unknown issues

• Exploratory tasks

• Tasks requiring adaptation

• Complex problem-solving

In [20]:
# Cell 11: Verify Created Files
"""
Verify that the CoT agent actually created the files
"""

project_path = CONFIG["sandbox_dir"] / "cot_fastapi_project"

if project_path.exists():
    console.print("\n[bold green]✓ Project directory exists[/bold green]\n")
    
    files_to_check = ["main.py", "requirements.txt", ".gitignore"]
    
    for filename in files_to_check:
        file_path = project_path / filename
        if file_path.exists():
            content = file_path.read_text()
            console.print(f"[cyan]File: {filename}[/cyan] ({len(content)} bytes)")
            
            # Show preview
            if filename == "main.py":
                syntax = Syntax(content[:500], "python", theme="monokai", line_numbers=True)
                console.print(syntax)
                if len(content) > 500:
                    console.print("[dim]... (truncated)[/dim]")
            else:
                console.print(f"[dim]{content[:200]}[/dim]")
                if len(content) > 200:
                    console.print("[dim]... (truncated)[/dim]")
            console.print()
        else:
            console.print(f"[red]✗ File not found: {filename}[/red]\n")
else:
    console.print("[red]✗ Project directory does not exist![/red]")

✓ Project directory exists

File: main.py (688 bytes)

   1 """                                                                                                           
   2 FastAPI Application Entry Point                                                                               
   3 """                                                                                                           
   4 from fastapi import FastAPI, HTTPException                                                                    
   5 from fastapi.middleware.cors import CORSMiddleware                                                            
   6 from pydantic import BaseModel                                                                                
   7                                                                                                               
   8 app = FastAPI(                                                                                                
   9     title="CoT API",                                                                                          
  10     description="Built with Chain of Thought",                                                                
  11     version="0.1.0"                                                                                           
  12 )                                                                                                             
  13                                                                                                               
  14 # CORS middleware                                                                                             
  15 app.add_middleware(                                                                                           
  16     CORSMiddleware,                                                                                           
  17     allow_origins=["*"],                                                                                      
  18     allow_credentials=True,                                                                                   
  19     allow_methods=["*"],                                                                                      
  20     allow_headers=["*"],                                                                                      
  21 )                                                                                                             
  22                                                                                                               
  23 @app.get("/")                                                                                                 
  24 async def root():                                                                                             
  25     return {"message": "Welcome                                                                               

... (truncated)

File: requirements.txt (139 bytes)

# Core dependencies
fastapi==0.104.1
uvicorn==0.24.0
pydantic==2.5.0
python-dotenv==1.0.0

# Testing
pytest==7.4.3
httpx==0.25.2

File: .gitignore (123 bytes)

# Python
__pycache__/
*.py
*$py.class
.Python
venv/
.venv/
ENV/

# IDE
.vscode/
.idea/

# Environment
.env
.env.local

# Conclusion

## Chain of Thought Agent Summary

We've implemented a **Chain of Thought (CoT) agent** that differs fundamentally from the ReAct pattern:

### Key Characteristics:

1. **Upfront Planning**: Generates complete plan before any execution
2. **Batch Execution**: Runs all steps sequentially without re-planning
3. **Efficiency**: Requires only 1 LLM call for planning vs N calls in ReAct
4. **Placeholder System**: Uses `PLACEHOLDER_CODE_FROM_STEP_N` to reference generated content

### Trade-offs:

**Advantages:**
- Faster execution (fewer LLM calls)
- Lower cost
- Simpler to debug (plan visible upfront)
- Better for deterministic tasks

**Disadvantages:**
- Cannot adapt if plan is wrong
- No error recovery
- Less suitable for exploratory tasks

### Best Use Cases:

Use CoT for:
- Project scaffolding
- Boilerplate generation
- Standard setup tasks
- Tasks with known, sequential steps

Use ReAct for:
- Debugging
- Complex problem-solving
- Exploratory tasks
- Situations requiring adaptation

---

**Next Steps**: Try combining both patterns - use CoT for planning, then switch to ReAct if errors occur!